In [1]:
from nilearn import maskers, datasets
from nilearn.interfaces.fmriprep import load_confounds_strategy
import json
import pickle

In [2]:
# 加载ALL脑图谱目标
# load AAL atlas template.
atlas = datasets.fetch_atlas_aal()
# Loading atlas image stored in 'maps'
atlas_filename = atlas['maps']

In [3]:
# 读取get_path.ipynb中获取的json文件
# load json file generated in get_path.ipynb
with open("./datasets.json", "r") as f:
    data = json.load(f)

In [ ]:
# 使用nilearn提取时间序列
# extract time series using nilearn module
for sub, ses in data.items():
    for op, runs in ses.items():
        if op == "soz":
            continue
        for run, items in runs.items():
            masker = maskers.NiftiLabelsMasker(labels_img=atlas_filename, mask_img=items["mask"], standardize=True, t_r=items["TR"],
                                    memory='nilearn_cache', memory_level=1, verbose=5)
            try:
                confounds_simple, sample_mask = load_confounds_strategy(items["bold"])
            except:
                continue
            items["time_series"] = masker.fit_transform(items["bold"])
            items["length"] = items["time_series"].shape[0]*items["TR"]
print(data["sub-292"]["ses-preop"]["run-01"]["time_series"].shape)

In [7]:
# 保存时间序列文件
# save time series as pickle file
with open("time_series.pkl", "wb") as f:
    pickle.dump(data, f)